# Code complexity analysis
We target programming languages found in the definition file provided by GitHub: https://github.com/github-linguist/linguist/blob/main/lib/linguist/languages.yml.


## Load dependencies

In [ ]:
import pandas as pd
import yaml
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

## Loading input CSV

In [ ]:
# input CSV
input_file = '../csvs/cloc_summary.csv'
raw_df = pd.read_csv(input_file)
raw_df

We copy this DataFrame to convert the data type of the columns to numeric

In [ ]:
df = raw_df.copy()
df['files'] = pd.to_numeric(df['files'], errors='coerce')
df['blank'] = pd.to_numeric(df['blank'], errors='coerce')
df['comment'] = pd.to_numeric(df['comment'], errors='coerce')
df['code'] = pd.to_numeric(df['code'], errors='coerce')

In [ ]:
programming_languages = set()
filepath = '../languages.yml'
if not os.path.isfile(filepath):
    print(f"Error: Languages YAML file not found at: {filepath}")
try:
    with open(filepath, 'r', encoding='utf-8') as f:
        data = yaml.safe_load(f)
        programming_languages = {lang for lang, details in data.items() 
                        if isinstance(details, dict) and details.get('type') == 'programming'}
        print(f"✅ Loaded {len(programming_languages)} programming languages from {filepath}.")
except Exception as e:
    print(f"Error reading or parsing YAML file: {e}")
programming_languages

In [ ]:
numeric_cols = ['files', 'blank', 'comment', 'code']

# Process Each Project Individually
processed_projects = []
for project_id in df['project_id'].unique():
    project_df = df[df['project_id'] == project_id]
    project_prog_df = project_df[project_df['language'].isin(programming_languages)].copy()
    
    if not project_prog_df.empty:
        new_sum = project_prog_df[numeric_cols].sum().to_dict()
        new_sum['project_id'] = project_id
        new_sum['language'] = 'SUM'
        sum_row = pd.DataFrame([new_sum])
        processed_projects.append(pd.concat([project_prog_df, sum_row]))
    else:
        zero_row_data = {
            'project_id': [project_id, project_id],
            'language': [pd.NA, 'SUM'],
            'files': [0, 0], 'blank': [0, 0], 'comment': [0, 0], 'code': [0, 0]
        }
        processed_projects.append(pd.DataFrame(zero_row_data))

# Assemble and Sort the Final DataFrame ---
df_final = pd.concat(processed_projects, ignore_index=True)

# Replace key function with a more robust sorting method
# Create a temporary column to define a clear sort priority for row types.
# Priority: 0 for NaN placeholder, 1 for actual languages, 2 for SUM rows.
conditions = [
    df_final['language'].isna(),
    df_final['language'] == 'SUM'
]
choices = [0, 2] # 0 for NaN, 2 for SUM
df_final['lang_order'] = np.select(conditions, choices, default=1) # 1 for everything else

# Perform a stable, multi-level sort and then remove the helper column
df_final = df_final.sort_values(
    by=['project_id', 'lang_order', 'language']
).drop(columns='lang_order').reset_index(drop=True)

# Display Result
df_final

In [ ]:
is_programming = df['language'].isin(programming_languages)
# if there is a project_id that has no programming languages, keep the project_id but with language counts as na and all counts as 0
df_filtered = df[is_programming].copy()

# Recalculate SUMs
# Group by project and sum the numeric columns to create the new SUM rows
numeric_cols = ['files', 'blank', 'comment', 'code']
df_sums = df_filtered.groupby('project_id')[numeric_cols].sum().reset_index()

# Add the 'language' column to mark these as SUM rows
df_sums['language'] = 'SUM'

# Combine and Format the Final DataFrame

# Append the new SUM rows to the filtered programming language rows
df_final = pd.concat([df_filtered, df_sums], ignore_index=True)

# Sort the results to keep projects grouped together, with SUM at the end.
df_final = df_final.sort_values(
    by=['project_id', 'language'],
    key=lambda col: col.apply(lambda x: 'ZZZ' if x == 'SUM' else x) # Trick to sort SUM last
).reset_index(drop=True)

# Reorder columns to match the original format
df_final = df_final[['project_id', 'language', 'files', 'blank', 'comment', 'code']]

In [ ]:
# save the new DataFrame to a new CSV
output_file = '../temp_data/revised_loc_analysis.csv'
df_final.to_csv(output_file, index=False)

## Statistical analysis

In [ ]:
filtered_df = df_final[df_final['language'] != 'SUM']
filtered_df

In [ ]:
# group by 'language' and aggregate the sum of columns
sums_per_language = filtered_df.groupby('language').agg({
    'files': 'sum',
    'code': 'sum'
}).reset_index()
# calculate the percentage of each language based on the 'code' column
sums_per_language['percentage'] = (sums_per_language['code'] / sums_per_language['code'].sum()) * 100
sums_per_language
sums_per_language.sort_values(by='code', ascending=False, inplace=True)
sums_per_language.head(20)

In [ ]:
keep_languages = ["JavaScript", "TypeScript", "C#", "PowerShell", "Ruby", "Java", "Python", "Go", "PHP", "CSS", "Rust", "Cython", "ANTLR", "C#", "Java"]
mask = ~sums_per_language["language"].isin(keep_languages)
sums_per_language.loc[mask, "language"] = "Other"
sums_per_language = sums_per_language.groupby("language", as_index=False).agg({"files": "sum", "code": "sum"})
sums_per_language = sums_per_language.sort_values(by='code', ascending=False)
sums_per_language

In [ ]:
# Calculate the Percentage Column
total_loc = sums_per_language['code'].sum()
sums_per_language['percentage'] = (sums_per_language['code'] / total_loc) * 100


# Add the "Total" Row
total_files = sums_per_language['files'].sum()
total_row = pd.DataFrame({
    'language': ['Total'],
    'files': [total_files],
    'code': [total_loc],
    'percentage': [100.0]
})
df_with_total = pd.concat([sums_per_language, total_row], ignore_index=True)

# Create a temporary key to define the sort order of row types.
# (0 = Language, 1 = Other, 2 = Total)
conditions = [
    df_with_total['language'] == 'Other',
    df_with_total['language'] == 'Total'
]
choices = [1, 2]
df_with_total['sort_key'] = np.select(conditions, choices, default=0)

# Sort by the key first (ascending), then by code (descending)
df_with_total = df_with_total.sort_values(
    by=['sort_key', 'code'],
    ascending=[True, False]
)

# Remove the temporary key
df_with_total = df_with_total.drop(columns='sort_key')

# Format for final display
display_df = df_with_total.copy()

display_df['files'] = display_df['files'].apply(lambda x: f"{int(x):,}")
display_df['code'] = display_df['code'].apply(lambda x: f"{int(x):,}")
display_df['percentage'] = display_df['percentage'].apply(lambda x: f"{x:.2f}%")

display_df = display_df.rename(columns={
    'language': 'Language',
    'files': 'Files',
    'code': 'LOC',
    'percentage': 'Percentage (%)'
})

# Display the final result
display_df

In [ ]:
# Convert to LaTeX Table

# Make the "Total" row bold for emphasis
# Find the index of the 'Total' row
total_row_index = display_df[display_df['Language'] == 'Total'].index

# Wrap each cell in that row with the \textbf{} command
for col in display_df.columns:
    display_df.loc[total_row_index, col] = display_df.loc[total_row_index, col].apply(
        lambda x: f"\\textbf{{{x}}}"
    )

# Generate the LaTeX table string from the DataFrame
latex_string = display_df.to_latex(
    index=False,
    caption="Summary of Language Distribution by Lines of Code (LOC).",
    label="tab:lang_dist",
    column_format="lrrr", # l=left, r=right alignment for 4 columns
    escape=False # Must be False to render the \textbf command correctly
)

# Print the result
print(latex_string)

# EDA

In [ ]:
filtered_df = df_final[df_final['language'] == 'SUM']
filtered_df

In [ ]:
# we save the DataFrame to a new CSV file
output_file = '../temp_data/revised_sum_per_repo.csv'
filtered_df.to_csv(output_file, index=False)

In [ ]:
# statistical summary of the 'code' column
loc_repos_summary = filtered_df['code'].describe()

# maximum
max_value = filtered_df['code'].max()

# minimum
min_value = filtered_df['code'].min()

# range (difference between max. and min.)
range_value = max_value - min_value

# median
median_value = filtered_df['code'].median()

# variance
variance_value = filtered_df['code'].var()

# standard deviation
std_deviation_value = filtered_df['code'].std()

# print results with readable format
pd.options.display.float_format = '{:.2f}'.format

# total number of repositories
total_repos = len(filtered_df)

# count of repositories with 1000 LOCs or less
repos_le_1000 = (filtered_df['code'] <= 1000).sum()

# count of repositories with 100 LOCs or less
repos_le_100 = (filtered_df['code'] <= 100).sum()

# calculate the percentages
percentage_le_1000 = (repos_le_1000 / total_repos) * 100
percentage_le_100 = (repos_le_100 / total_repos) * 100

# display the new results
print(f"\n% of repos with <= 1000 LOCs: {percentage_le_1000:.2f}%")
print(f"% of repos with <= 100 LOCs: {percentage_le_100:.2f}%")

# show the results
print("Statistical summary:")
print(loc_repos_summary)
print("\nMax.:", max_value)
print("Min.:", min_value)
print("Range:", range_value)
print("Median:", median_value)
print("Variance:", variance_value)
print("Standard deviation:", std_deviation_value)

In [ ]:
# extract the 'code' column
loc_repos_data = filtered_df['code']

# calculate the IQR
Q1 = loc_repos_data.quantile(0.25)
Q3 = loc_repos_data.quantile(0.75)
IQR = Q3 - Q1

# calculate the percentage of values within the IQR limits
iqr_lower_bound = Q1 - 1.5 * IQR
iqr_upper_bound = Q3 + 1.5 * IQR
iqr_lower_bound = max(0, iqr_lower_bound)
percentage_in_iqr = ((loc_repos_data >= iqr_lower_bound) & (loc_repos_data <= iqr_upper_bound)).mean() * 100

# Probability Density Function (PDF)
fig_pdf, ax_pdf = plt.subplots(figsize=(10, 6))
ax_pdf.hist(loc_repos_data, bins=20, density=True, alpha=0.7, color='blue', edgecolor='black')
ax_pdf.set_xlabel('LOC', fontsize=12)
ax_pdf.set_ylabel('Probability', fontsize=12)
ax_pdf.set_title('Probability density function (PDF) for the lines of code (LOC)', fontsize=14)
ax_pdf.tick_params(axis='both', labelsize=10)
ax_pdf.grid(axis='y', linestyle='--', alpha=0.7)
ax_pdf.text(0.2, 0.7, 'Mean: {:.2f}'.format(loc_repos_data.mean()), transform=ax_pdf.transAxes, fontsize=10, color='purple')
ax_pdf.text(0.2, 0.675, 'Standard deviation: {:.2f}'.format(loc_repos_data.std()), transform=ax_pdf.transAxes, fontsize=10, color='purple')
ax_pdf.text(0.2, 0.650, f'Data in IQR: {percentage_in_iqr:.2f}%', transform=ax_pdf.transAxes, fontsize=10, color='purple')
ax_pdf.axvline(iqr_lower_bound, color='green', linestyle='--', label='IQR lower bound')
ax_pdf.axvline(iqr_upper_bound, color='green', linestyle='--', label='IQR upper bound')
ax_pdf.legend(loc='upper right', fontsize=10)
pdf_file = '../paper/figs/revised_pdf_loc_repos.pdf'
fig_pdf.savefig(pdf_file, format='pdf', dpi=300)

# Cumulative Density Function (CDF)
fig_cdf, ax_cdf = plt.subplots(figsize=(10, 6))
sorted_data = np.sort(loc_repos_data)
y = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
ax_cdf.plot(sorted_data, y, marker='.', linestyle='-', color='red', markersize=4, markeredgecolor='black')
ax_cdf.set_xlabel('LOC', fontsize=12)
ax_cdf.set_ylabel('Cumulative probability', fontsize=12)
ax_cdf.set_title('Cumulative density function (CDF) for the lines of code (LOC)', fontsize=14)
ax_cdf.tick_params(axis='both', labelsize=10)
ax_cdf.grid(axis='y', linestyle='--', alpha=0.7)
ax_cdf.text(0.2, 0.2, '25th percentile: {:.2f}'.format(np.percentile(sorted_data, 25)), transform=ax_cdf.transAxes, fontsize=10, color='purple')
ax_cdf.text(0.2, 0.175, '75th percentile: {:.2f}'.format(np.percentile(sorted_data, 75)), transform=ax_cdf.transAxes, fontsize=10, color='purple')
ax_cdf.text(0.2, 0.150, f'Data in IQR: {percentage_in_iqr:.2f}%', transform=ax_cdf.transAxes, fontsize=10, color='purple')
ax_cdf.axvline(iqr_lower_bound, color='blue', linestyle='--', label='IQR lower bound')
ax_cdf.axvline(iqr_upper_bound, color='blue', linestyle='--', label='IQR upper bound')
ax_cdf.legend(loc='lower right', fontsize=10)
cdf_file = '../paper/figs/revised_cdf_loc_repos.pdf'
fig_cdf.savefig(cdf_file, format='pdf', dpi=300)

# Display the plots
plt.show()


In [ ]:
# create 1 subplot
fig, ax2 = plt.subplots(figsize=(8, 6))

# calculate and graph the CDF
sorted_data = np.sort(loc_repos_data)
y = np.arange(1, len(sorted_data) + 1) / len(sorted_data)
ax2.plot(sorted_data, y, color='blue', linewidth=2)
ax2.set_xlabel('LOC', fontsize=22)  # Increased font size
ax2.set_ylabel('Cumulative probability', fontsize=22)  # Increased font size
ax2.set_title('LOC cumulative distribution function (CDF)', fontsize=22)  # Increased font size

# customize graphs
ax2.tick_params(axis='both', labelsize=20)  # Increased tick label font size
ax2.grid(axis='y', linestyle='--', alpha=0.7)

# adjust scale to log
ax2.set_xscale('log')

# adjust spacing between subplots and labels
plt.tight_layout()

# save the plots as image files
df_file = '../paper/figs/revised_cdf_loc_repos_log.pdf'
plt.savefig(df_file, dpi=300)  # adjust DPI for high res. outputs

# shows the plots
plt.show()

In [ ]:
# generate DFs from input CSVs
input_file_1 = '../temp_data/revised_loc_analysis.csv'
input_file_2 = '../temp_data/event_counts_per_repo.csv'

In [ ]:
df1 = pd.read_csv(input_file_1)
columns_to_remove = ['language', 'files', 'blank', 'comment']
df1 = df1.drop(columns=columns_to_remove)
df1

In [ ]:
# removing outliers from df1
loc_data = df1['code']

# calculate the interquartile range (IQR)
Q1 = loc_data.quantile(0.25)
Q3 = loc_data.quantile(0.75)
IQR = Q3 - Q1

# calculate the percentage of values within the IQR limits
iqr_lower_bound = Q1 - 1.5 * IQR
iqr_upper_bound = Q3 + 1.5 * IQR
iqr_lower_bound = max(0, iqr_lower_bound)
print(f"Lower bound: {iqr_lower_bound}, Upper bound: {iqr_upper_bound}")

df1 = df1[(df1['code'] >= iqr_lower_bound) & (df1['code'] <= iqr_upper_bound)]
df1

In [ ]:
df2 = pd.read_csv(input_file_2)
df2

In [ ]:
# we remove outliers from df2
event_count_data = df2['count']

# calculate the interquartile range (IQR)
Q1 = event_count_data.quantile(0.25)
Q3 = event_count_data.quantile(0.75)
IQR = Q3 - Q1

# calculate the percentage of values within the IQR limits, ensuring the lower limit is not less than 0
iqr_lower_bound = Q1 - 1.5 * IQR
iqr_upper_bound = Q3 + 1.5 * IQR
iqr_lower_bound = max(0, iqr_lower_bound)
print(f"Lower bound: {iqr_lower_bound}, Upper bound: {iqr_upper_bound}")

df2 = df2[(df2['count'] >= iqr_lower_bound) & (df2['count'] <= iqr_upper_bound)]
df2

In [ ]:
merged_df = pd.merge(df1, df2, left_on='project_id', right_on='project_id', how='inner')
merged_df = merged_df.drop(columns=['project_id'])
merged_df

In [ ]:
# calculate the correlation between 'code' and 'count'
correlation = merged_df['code'].corr(merged_df['count'])

In [ ]:
# create a scatter plot
plt.figure(figsize=(8, 6))
sns.set_context("notebook", font_scale=1.5)  # Adjust font_scale to increase font size
sns.scatterplot(data=merged_df, x='code', y='count')
plt.title(f'Number of event triggers vs. LOC (correlation: {correlation:.2f})', fontsize=18)
plt.xlabel('LOC' , fontsize=16)
plt.ylabel('No. of event triggers', fontsize=16)

# save the plot as a PDF
scatter_plot_pdf_file = '../paper/figs/revised_scatter_plot_event_triggers_vs_loc.pdf'
plt.savefig(scatter_plot_pdf_file, format='pdf', dpi=300)

# display the plot
plt.show()